# Firth logistic regression and baseline characteristics
##### Updated 07/03/2024
##### Selin Kubali

#### Goal:
Calculate Firth logistic regression for missense and LOF variants for all genes; calculate median and means for clinical, demographic, and variant characteristics.

#### Required inputs

Lifelines files - Stored in */selected_genes/hcm/cox_model_data*. 


##### Load packages

In [ ]:
!pip install firthlogist
!pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
from firthlogist import FirthLogisticRegression
from statsmodels.stats.multitest import multipletests

In [6]:
gene_list = ['ACTC1', 'ACTN2', 'ALPK3', 'DES','FLNC','MYBPC3','MYH6','MYH7','MYL2','MYL3','PLN','PTPN11','TNNI3','TNNT2','TPM1','TTR']

##### Firth logistic regression

In [ ]:
deleterious_vals = {}
deleterious_odds = {}
missense_vals = {}
missense_odds = {} 
deleterious_variant_nums = {}
missense_variant_nums = {}



def logistic_regression(gene, dir_path = 'selected_genes/hcm/cox_model_data'):
    # read in file
    lifelines_data = pd.read_csv(f'/mnt/project/{dir_path}/{gene}_with_generated_data.csv', dtype={
            'is_family_hist':'boolean',
            'is_HCM':'boolean'
            })
    lifelines_data = lifelines_data[['is_HCM','sex','is_family_hist', 'age', 'prs_score', 'Consequence', 'am_pathogenicity']]                      
    lifelines_data = lifelines_data.dropna(subset = ['is_HCM','sex','is_family_hist', 'age', 'prs_score'])
    lifelines_data['val'] = 1
    consequences = lifelines_data.pivot(columns = ['Consequence'], values = 'val')[['deleterious','missense_variant','synonymous_variant']]
    lifelines_data = lifelines_data.drop(['Consequence','val'],axis=1)
    consequences = consequences.fillna(value=0)
    lifelines_data['index'] = lifelines_data.index
    consequences['index'] = consequences.index
    lifelines_data = pd.merge(consequences, lifelines_data)
    # control for sex, family history, age, and PRS score
    X=lifelines_data[['sex','is_family_hist', 'age', 'prs_score', 'deleterious','missense_variant']].values
    y = lifelines_data['is_HCM'].values
    feature_names = ['sex','is_family_hist', 'age', 'prs_score', 'deleterious','missense_variant']
    fl = FirthLogisticRegression()
    fl.fit(X, y)
    
    # extract LOF information
    deleterious_ci_string = ''.join([str("{:.{}g}".format(np.exp(fl.coef_[4]), 3)),' (',
    str("{:.{}g}".format(np.exp(fl.ci_[4][0]), 3)), '-',
    str("{:.{}g}".format(np.exp(fl.ci_[4][1]), 3)),')'])
   
    # extract missense information

    missense_ci_string = ''.join([str("{:.{}g}".format(np.exp(fl.coef_[5]), 3)), ' (',
    str("{:.{}g}".format(np.exp(fl.ci_[5][0]), 3)), '-',  
    str("{:.{}g}".format(np.exp(fl.ci_[5][1]), 3)), ')'])

    
    fl.summary(xname=feature_names)
    deleterious_vals.update({gene:fl.pvals_[4]})
    deleterious_odds.update({gene:deleterious_ci_string})
    missense_vals.update({gene:fl.pvals_[5]})
    missense_odds.update({gene:missense_ci_string})
    deleterious_variant_nums.update({gene:len(lifelines_data[lifelines_data['deleterious'] == 1])})
    missense_variant_nums.update({gene:len(lifelines_data[lifelines_data['missense_variant'] == 1])})


for gene in gene_list:
    print(gene)
    logistic_regression(gene)



##### LOF variants

In [8]:
p_adjusted = multipletests(list(deleterious_vals.values()), alpha=0.05, method='fdr_bh') # correct for multiple comparisons
p_adjusted_sigs = []
for i in p_adjusted[1]:
    p_adjusted_sigs.append("{:.{}g}".format(i, 3))

updated_dict = {key: new_p_val for key, new_p_val in zip(deleterious_vals.keys(), p_adjusted_sigs)}
deleterious_df = pd.DataFrame({'Odds': deleterious_odds, 'LOF p-values': updated_dict, 'Number of LOF variants': deleterious_variant_nums} )
deleterious_df

,Odds,Deleterious p-values,Number of deleterious variants
ACTC1,17.6 (1.99-65.1),0.0534,68
ACTN2,2.14 (0.245-7.68),0.641,464
ALPK3,6.15 (2.86-11.4),0.000393,969
DES,1.04 (0.00825-7.09),0.963,304
FLNC,1.51 (0.173-5.4),0.705,691
MYBPC3,59.7 (40.5-85.2),1.66e-43,454
MYH6,0.694 (0.0794-2.47),0.705,1567
MYH7,1.7 (0.357-4.79),0.641,990
MYL2,3.12 (0.874-7.68),0.17,771
MYL3,8.18 (0.0644-58.7),0.48,39


##### Missense variants

In [9]:
p_adjusted = multipletests(list(missense_vals.values()), alpha=0.1, method='fdr_bh') #correct for multiple comparisons
p_adjusted_sigs = []
for i in p_adjusted[1]:
    p_adjusted_sigs.append("{:.{}g}".format(i, 3))

updated_dict = {key: new_p_val for key, new_p_val in zip(missense_vals.keys(), p_adjusted_sigs)}
p_val_true_dict = {key: new_p_val for key, new_p_val in zip(missense_vals.keys(), p_adjusted[0])}
missense_df = pd.DataFrame({'Odds': missense_odds, 'Missense p-values': updated_dict, 'Number of missense variants': missense_variant_nums} )
missense_df

,Odds,Missense p-values,Number of missense variants
ACTC1,10.1 (1.15-37),0.0728,107
ACTN2,1.31 (0.641-2.34),0.526,4829
ALPK3,1.6 (1.02-2.38),0.0728,9667
DES,0.719 (0.0823-2.56),0.668,1382
FLNC,1.85 (1.24-2.66),0.0153,10013
MYBPC3,2.25 (1.45-3.32),0.0049,7456
MYH6,0.863 (0.439-1.5),0.668,8254
MYH7,7.68 (5.59-10.3),4.22e-23,4278
MYL2,3.2 (1.32-6.37),0.0305,1344
MYL3,2.56 (0.719-6.31),0.187,904


##### Calculate demographic and clinical characteristics

In [15]:

# read in file: demographic and clinical characteristics are the same for all genes
lifelines_data = pd.read_csv(f'/mnt/project/selected_genes/hcm/cox_model_data/ACTN2_with_generated_data.csv', dtype={
        'is_family_hist':'boolean',
        'is_hcm':'boolean'
        })

lifelines_data['prs_score'] = (lifelines_data['prs_score'] - lifelines_data['prs_score'].min()) / (lifelines_data['prs_score'].max() - lifelines_data['prs_score'].min())  # normalize PRS  

lifelines_data = lifelines_data[['is_HCM', 'duration', 'prs_score', 'sex', 'is_family_hist', 'is_AF', 'is_HTN']]
lifelines_data_hcm = lifelines_data[lifelines_data['is_HCM'] == 1]
lifelines_data_no_hcm = lifelines_data[lifelines_data['is_HCM'] == 0]

In [16]:
significant_figures = 2
medians = pd.DataFrame({})
dict_list = []
for cols in ['duration', 'prs_score']:
    col_list = {}
    for df, df_name in zip([lifelines_data, lifelines_data_hcm, lifelines_data_no_hcm], ['Full Cohort' ,'With HCM', 'Without HCM']):
        col = df[cols]
        col = col[~np.isnan(col)]
        percentiles = np.percentile(col, [25, 50, 75]) # find median and IQR
        formatted_numbers = []

        # format information
        for num in percentiles:
            formatted_numbers.append("{:.{}g}".format(num, significant_figures))
        string = str(formatted_numbers[1])+' ('+str(formatted_numbers[0])+'-'+str(formatted_numbers[2])+')'
        col_list[df_name] = string
    dict_list.append(col_list)
    
df = pd.DataFrame(dict_list, index = ['Duration', 'Normalized PRS score'])
df

,Full Cohort,With HCM,Without HCM
Duration,71 (64-77),63 (55-72),71 (64-77)
Normalized PRS score,0.43 (0.36-0.51),0.49 (0.41-0.57),0.43 (0.36-0.51)


In [12]:
sums = pd.DataFrame({})

dict_list = []
for cols in ['sex', 'is_family_hist', 'is_AF', 'is_HTN']:
    col_list = {}
    for df, df_name in zip([lifelines_data, lifelines_data_hcm, lifelines_data_no_hcm], ['Full Cohort' ,'With HCM (Num = 751)', 'Without HCM (Num = 501608)']):
        col = df[cols]
        col = col[~np.isnan(col)]
        formatted_numbers = []

        # format information
        string = str("{:.{}g}".format(col.sum(), 7)) + ' (' + str("{:.{}g}".format(col.mean(), 2)) + '%)'
        col_list[df_name] = string
    dict_list.append(col_list)
    
df = pd.DataFrame(dict_list, index = ['Number Male', 'Family History of CAD', 'Atrial Fibrillation', 'Hypertension'])
df

,Full Cohort,With HCM (Num = 751),Without HCM (Num = 501608)
Number Male,229063 (0.46%),481 (0.64%),228582 (0.46%)
Family History of CAD,223409 (0.44%),436 (0.58%),222973 (0.44%)
Atrial Fibrillation,22872 (0.046%),217 (0.29%),22655 (0.045%)
Hypertension,200298 (0.4%),553 (0.74%),199745 (0.4%)


In [ ]:
# combine information from all genes
full_df = pd.DataFrame({})
gene_list = ['ACTC1','ACTN2','ALPK3','DES','FLNC','MYBPC3','MYH6','MYH7','MYL2','MYL3','PLN','PTPN11','TNNI3','TNNT2','TPM1','TTR']
for gene in gene_list:
    gene_data = pd.read_csv(f'/mnt/project/selected_genes/hcm/cox_model_data/{gene}_with_generated_data.csv', dtype={
            })
    print(gene)

            
    if 'revel_max' not in gene_data.columns:
        gene_data = gene_data[['am_pathogenicity','spliceai_ds_max','cadd_raw_score', 'phylop', 'faf_max','Consequence', 'is_HCM']]
    else:
        gene_data = gene_data[['am_pathogenicity', 'spliceai_ds_max', 'revel_max', 'cadd_raw_score', 'phylop', 'faf_max', 'Consequence', 'is_HCM']]
        gene_data['revel_max'] = (gene_data['revel_max'] - gene_data['revel_max'].min()) / (gene_data['revel_max'].max() - gene_data['revel_max'].min())     
        
    gene_data = gene_data[gene_data['Consequence'] == 'missense_variant']

    gene_data = gene_data.drop(['Consequence'], axis = 1)
    gene_data['am_pathogenicity'] = (gene_data['am_pathogenicity'] - gene_data['am_pathogenicity'].min()) / (gene_data['am_pathogenicity'].max() - gene_data['am_pathogenicity'].min())     
    gene_data['cadd_raw_score'] = (gene_data['cadd_raw_score'] - gene_data['cadd_raw_score'].min()) / (gene_data['cadd_raw_score'].max() - gene_data['cadd_raw_score'].min())     
    gene_data['phylop'] = (gene_data['phylop'] - gene_data['phylop'].min()) / (gene_data['phylop'].max() - gene_data['phylop'].min())     


    full_df = pd.concat([full_df,gene_data])

##### Calculate variant characteristics

In [20]:
lifelines_data = full_df[['is_HCM',  'am_pathogenicity', 'spliceai_ds_max', 'revel_max', 'cadd_raw_score', 'phylop', 'faf_max',]]
lifelines_data_hcm = full_df[full_df['is_HCM'] == 1]
lifelines_data_no_hcm = full_df[full_df['is_HCM'] == 0]

significant_figures = 2
medians = pd.DataFrame({})
dict_list = []
for cols in [ 'am_pathogenicity', 'spliceai_ds_max', 'revel_max', 'cadd_raw_score', 'phylop', 'faf_max',]:
    col_list = {}
    for df, df_name in zip([lifelines_data, lifelines_data_no_hcm, lifelines_data_hcm], ['Full Cohort' , 'Without HCM', 'With HCM']):
        col = df[cols]
        col = col[~np.isnan(col)]
        percentiles = np.percentile(col, [25, 50, 75]) # find medians and IQR
        formatted_numbers = []

        # format
        for num in percentiles:
            formatted_numbers.append("{:.{}g}".format(num, significant_figures))
        string = str(formatted_numbers[1])+' ('+str(formatted_numbers[0])+'-'+str(formatted_numbers[2])+')'
        col_list[df_name] = string
    dict_list.append(col_list)
    
df = pd.DataFrame(dict_list, index = ['AlphaMissense', 'SpliceAI', 'REVEL', 'CADD', 'PhyloP', 'Pop freq'])
df

,Full Cohort,Without HCM,With HCM
AlphaMissense,0.26 (0.065-0.54),0.26 (0.065-0.54),0.35 (0.091-0.74)
SpliceAI,0 (0-0.01),0 (0-0.01),0 (0-0.01)
REVEL,0.39 (0.19-0.68),0.39 (0.19-0.68),0.61 (0.34-0.74)
CADD,0.71 (0.56-0.82),0.71 (0.56-0.82),0.76 (0.63-0.84)
PhyloP,0.85 (0.68-0.98),0.85 (0.68-0.98),0.88 (0.71-0.99)
Pop freq,8.9e-05 (2.3e-05-0.00046),8.9e-05 (2.3e-05-0.00046),3.2e-05 (6.1e-06-0.00017)
